**transcription = merubah suara anotasi menjadi teks**

suara dibagi perdetik, perdetiknya akan dibaca oleh CNN



In [21]:
import pandas as pd
data = pd.read_csv('sentimen_kalimat.csv').values
data

array([['eh ihh uhh ehh uhh ohh eeh uhhh oooh uuh ahhh ahhh eeh uh iih ah ohhh oooh aah aah ahhh ooh iiih oh',
        0],
       ['iiih aah uhh ohhh eh aaah ehh ehh uuuh ihh ihhh ehh ihh uuuh uhhh ohh aah ihh ohh eeeh eh uhh ooh ehhh aah ohh uh ahhh',
        2],
       ['ooh uhhh aah ah ah iiih ohh oooh', 1],
       ...,
       ['ih uhhh ahh ihh ohh ihhh aah ihh uuuh ih uhh iih eeh uuh ihhh ehh iiih ohhh eeeh iih ohhh aah aah uuuh iiih oooh ahhh aah ohhh',
        2],
       ['eeh ahhh iih ooh uhhh eeh uh ohh eeh eeh uuuh ooh uh uhhh uhh ohh oooh uuuh eh iiih ah uh aah eeeh ihh',
        0],
       ['ehh ohh iih ooh ehh uuh oooh oh', 0]], dtype=object)

In [22]:
x = data [:,0]
y = data [:,1]. astype (int)

In [23]:
#mencari kata terpanjang di file sentimen kalimat
max_kata = max([len(kalimat.split(' ')) for kalimat in x])
max_kata

30

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer #merubah huruf menjadi bentuk angka
from tensorflow.keras.preprocessing.sequence import pad_sequences #menyamakan jumlah kata

In [25]:
#mendeklarasikan token ah dan uh
token_kal = Tokenizer()

#train
token_kal.fit_on_texts(x)

#text ke angka
v_kal = token_kal.texts_to_sequences(x)
v_kal

[[12,
  4,
  20,
  16,
  20,
  10,
  8,
  22,
  24,
  18,
  13,
  13,
  8,
  6,
  5,
  23,
  17,
  24,
  2,
  2,
  13,
  7,
  9,
  19],
 [9,
  2,
  20,
  17,
  12,
  1,
  16,
  16,
  3,
  4,
  15,
  16,
  4,
  3,
  22,
  10,
  2,
  4,
  10,
  14,
  12,
  20,
  7,
  11,
  2,
  10,
  6,
  13],
 [7, 22, 2, 23, 23, 9, 10, 24],
 [10,
  21,
  13,
  2,
  24,
  15,
  24,
  17,
  24,
  17,
  3,
  3,
  11,
  14,
  6,
  22,
  2,
  1,
  23,
  10,
  3,
  15,
  10,
  22,
  17,
  19],
 [2, 25, 11, 8, 21],
 [13, 5, 21, 14, 18, 2],
 [25, 16, 3, 17, 21, 8, 17, 19, 1],
 [5, 5, 25, 24, 24, 17, 23, 5],
 [18, 2, 9, 6, 22, 12, 22],
 [6,
  24,
  14,
  18,
  25,
  20,
  9,
  4,
  20,
  12,
  14,
  23,
  13,
  19,
  6,
  13,
  1,
  7,
  14,
  5,
  24,
  21],
 [13,
  9,
  23,
  16,
  22,
  1,
  16,
  9,
  13,
  4,
  5,
  23,
  20,
  23,
  10,
  9,
  8,
  15,
  16,
  6,
  25,
  21,
  5],
 [13,
  16,
  22,
  3,
  17,
  4,
  4,
  19,
  6,
  23,
  7,
  13,
  12,
  4,
  11,
  12,
  14,
  6,
  18,
  19,
  4,
  4,
  9,

In [26]:
v_kal = pad_sequences(v_kal, maxlen=max_kata, padding='pre')

In [27]:
print(v_kal.shape)

(5000, 30)


In [28]:
total_kata_kalimat = len(token_kal.word_index)
print(total_kata_kalimat)

25


In [31]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU #Timedistributed untuk multiple output san input
from keras.layers import Embedding, Bidirectional 

model = Sequential()
model.add(Embedding(total_kata_kalimat+1, 16, input_shape = v_kal.shape[1:], input_length = v_kal.shape[1])) #32 merupakan jumlah selnya (setelah proses 32 kali baru berjalan)
model.add(Bidirectional (GRU(32))) #time distributed menjamin jumlah output sesuai input 
model.add(Dense(8, activation='relu')) #+1 untuk menyiapkan 1 kolom
model.add(Dense(3, activation='softmax')) #+1 untuk menyiapkan 1 kolom
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 30, 16)            416       
                                                                 
 bidirectional_5 (Bidirectio  (None, 64)               9600      
 nal)                                                            
                                                                 
 dense_10 (Dense)            (None, 8)                 520       
                                                                 
 dense_11 (Dense)            (None, 3)                 27        
                                                                 
Total params: 10,563
Trainable params: 10,563
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.fit(v_kal, y, batch_size=2, epochs=10, validation_split=0.2)

Epoch 1/10
2000/2000 [==============================] - 32s 15ms/step - loss: 0.6157 - accuracy: 0.7195 - val_loss: 0.3016 - val_accuracy: 0.8800
Epoch 2/10
2000/2000 [==============================] - 28s 14ms/step - loss: 0.2151 - accuracy: 0.9222 - val_loss: 0.1757 - val_accuracy: 0.9350
Epoch 3/10
2000/2000 [==============================] - 28s 14ms/step - loss: 0.1778 - accuracy: 0.9360 - val_loss: 0.1526 - val_accuracy: 0.9360
Epoch 4/10
2000/2000 [==============================] - 28s 14ms/step - loss: 0.1253 - accuracy: 0.9603 - val_loss: 0.2933 - val_accuracy: 0.8920
Epoch 5/10
2000/2000 [==============================] - 28s 14ms/step - loss: 0.1299 - accuracy: 0.9592 - val_loss: 0.2275 - val_accuracy: 0.9080
Epoch 6/10
2000/2000 [==============================] - 29s 14ms/step - loss: 0.1085 - accuracy: 0.9645 - val_loss: 0.0523 - val_accuracy: 0.9920
Epoch 7/10
2000/2000 [==============================] - 30s 15ms/step - loss: 0.1015 - accuracy: 0.9655 - val_loss: 0.1012 -